# Data Processing

This notebook is meant for all data cleaning and processing. Feel free to add or change sections.

Some Todos:
- `kernel_receivals`
    - Remove all rows where `rm_id`'s don't exist in `prediction_mapping`.
    - Remove all rows in `kernel_purchase_orders` with `quantity  <= 0`.
    - Remove `product_id` and `receival_status`.
    - Remove `batch_id`. It appears only from 2015 and onwards --> Poor data quality.
- `kernel_purchase_orders`
    - Remove `product_id`. There are multiple `rm_id`'s for each product_id, therefore it is useless here. We will know the rm_id after the dataset is joined with kernel_receivals based on purchase_order_id and/or purchase_order_item_no.
    - Remove `unit` and `unit_id`. Almost all entries have KG and the ones that do not are simply NaN and can be assumed to be KG as well.
    - Remove `status_id`. The status is duplicated in `status`.
    - Remove `product_version`. It is just set to 1 for all orders up until 2016 --> Poor data quality.
- `extended_materials`
    - Remove all rows where `rm_id`'s don't exist in `prediction_mapping`.
    - Remove `raw_material_alloy`, `product_id`.
    - There are a good deal of entries where `stock_location` is prefixed with DELETED followed by a date. It would be useful to check if there are differences in deliveries before and after that date. My hypothesis is that after it is deleted, there are no more deliveries for that rm_id. If this is a case, a boolean feature `is_deleted` would be powerful. Before inserting a DELETED=TRUE tag for an rm_id, check that there are no other listings of the same rm_id's that do not have DELETED in the stock_location variable. If there are other non-DELETED locations, then it may be that the stock location has simply moved but is still active. Be sure to also make a `deleted_date` feature as well. Then it is possible to create a `days_since_deleted = (forecast_date - deleted_date)` feature.
    - Remove `stock_location`, but only after making the `is_deleted` boolean and`date_deleted`.
- `extended_transportation`
    - Remove all rows where `rm_id`'s don't exist in `prediction_mapping`.
    - Remove `product_id`.
    - Remove `unit_status`. Almost all entries are 'Tranferred', a some are 'Accepted' and a couple are 'Pending'.
    - Remove `wood`, `ironbands`, `plastic`, `water`, `ice`, `other`, `chips`, `packaging`, `cardboard`. These have very few entries.



## Setup

### Imports

In [2]:
import pandas as pd
from pathlib import Path
from tabulate import tabulate
from typing import Dict

### Helper Functions

In [3]:
def load_data(filename, folder="1_raw"):
    """
    Load data from a CSV file in a subfolder of the project's 'data' directory.
    This version is adjusted to work even if the notebook is run from a subfolder.

    Parameters
    ----------
    filename : str
        The name of the file to load, including the extension (e.g., "data.csv").
    folder : str, optional
        The subfolder within 'data' to load from. Defaults to "1_raw".
    """
    try:
        # Go up one level from the current working directory to find the project root
        PROJECT_ROOT = Path.cwd().parent

        file_path = PROJECT_ROOT / "data" / folder / filename

        df = pd.read_csv(file_path, sep=",")

        print(f"Data loaded successfully from {file_path}")
        return df
    except FileNotFoundError:
        print(f"Error: The file was not found at {file_path}")
        return None
    except Exception as e:
        print(f"An error occurred while loading the file: {e}")
        return None


def save_data(df, filename, folder="2_interim"):
    """
    Save a dataframe to a CSV file in a subfolder of the project's 'data' directory.

    This function automatically creates the destination folder if it doesn't exist.

    Parameters
    ----------
    df : pandas.DataFrame
        The dataframe to save.
    filename : str
        The name for the output file, including the extension (e.g., "processed_orders.csv").
    folder : str, optional
        The subfolder within 'data' to save to. Defaults to "2_interim".
    """
    try:
        PROJECT_ROOT = Path.cwd().parent
        save_dir = PROJECT_ROOT / "data" / folder
        save_dir.mkdir(parents=True, exist_ok=True)

        # The full filename, including extension, is now expected
        file_path = save_dir / filename

        df.to_csv(file_path, sep=",", index=False)

        print(f"Data saved successfully to {file_path} ✅")

    except Exception as e:
        print(f"An error occurred while saving the file: {e}")


def generate_feature_presence_table(datasets: Dict[str, pd.DataFrame]):
    """
    Analyzes a dictionary of pandas DataFrames to determine the presence and
    fill grade of features (columns) across all of them. It then prints a
    formatted table summarizing this information.

    Args:
        datasets (Dict[str, pd.DataFrame]): A dictionary where keys are string
            names for the DataFrames and values are the pandas DataFrames
            themselves.
    """
    if not isinstance(datasets, dict) or not all(
        isinstance(df, pd.DataFrame) for df in datasets.values()
    ):
        print("Error: Input must be a dictionary of pandas DataFrames.")
        return

    # --- 1. Identify all unique features across all DataFrames ---
    all_features = sorted(
        list(set(feature for df in datasets.values() for feature in df.columns))
    )

    # --- 2. Build the presence and fill grade data for each DataFrame ---
    presence_data = []
    for name, df in datasets.items():
        row = {"DataFrame Name": name}
        for feature in all_features:
            if feature in df.columns:
                series = df[feature]
                total_count = len(series)

                if total_count > 0:
                    non_nan_count = series.count()  # .count() excludes NaNs
                    fill_grade = (non_nan_count / total_count) * 100
                    row[feature] = f"✅ {fill_grade:.1f}%"
                else:
                    # Handle empty DataFrames/Series
                    row[feature] = "✅ 100.0%"
            else:
                # Feature is not present in this DataFrame
                row[feature] = ""
        presence_data.append(row)

    # --- 3. Create, transpose, and format the DataFrame for display ---
    if not presence_data:
        print("No data to display.")
        return

    presence_df = pd.DataFrame(presence_data)
    presence_df = presence_df.set_index("DataFrame Name")

    # Transpose the DataFrame so features are rows and datasets are columns
    presence_df_transposed = presence_df.T
    presence_df_transposed.index.name = "Feature Name"

    # --- 4. Print the final table using tabulate ---
    print("--- Feature Presence & Fill Grade Across All DataFrames ---")
    print(
        tabulate(
            presence_df_transposed, headers="keys", tablefmt="grid", stralign="center"
        )
    )
    print("\n" * 2)  # Add some space after the table

### Data Loading

In [4]:
# Kernel files
df_kpo = load_data("kernel_purchase_orders.csv")
df_kr = load_data("kernel_receivals.csv")

# Extended files
df_em = load_data("extended_materials.csv")
df_et = load_data("extended_transportation.csv")

Data loaded successfully from /home/kfkh/Code/tdt4173-course-project/data/1_raw/kernel_purchase_orders.csv
Data loaded successfully from /home/kfkh/Code/tdt4173-course-project/data/1_raw/kernel_receivals.csv
Data loaded successfully from /home/kfkh/Code/tdt4173-course-project/data/1_raw/extended_materials.csv
Data loaded successfully from /home/kfkh/Code/tdt4173-course-project/data/1_raw/extended_transportation.csv


## Filtering

In [5]:
# Order datasets for iteration
datasets = {
    "kernel_receivals": df_kr,
    "kernel_purchase_orders": df_kpo,
    "extended_materials": df_em,
    "extended_transportation": df_et,
}
generate_feature_presence_table(datasets)

--- Feature Presence & Fill Grade Across All DataFrames ---
+--------------------------+--------------------+--------------------------+----------------------+---------------------------+
|       Feature Name       |  kernel_receivals  |  kernel_purchase_orders  |  extended_materials  |  extended_transportation  |
+==========================+====================+==========================+======================+===========================+
|         batch_id         |      ✅ 52.8%      |                          |                      |         ✅ 52.8%          |
+--------------------------+--------------------+--------------------------+----------------------+---------------------------+
|        cardboard         |                    |                          |                      |          ✅ 6.4%          |
+--------------------------+--------------------+--------------------------+----------------------+---------------------------+
|          chips           |                   

In [6]:
# Load the mapping file to get the relevant rm_id's
df_map = load_data("prediction_mapping.csv")
relevant_rm_ids = df_map["rm_id"].unique()
print(f"Found {len(relevant_rm_ids)} unique rm_id's to focus on.")

Data loaded successfully from /home/kfkh/Code/tdt4173-course-project/data/1_raw/prediction_mapping.csv
Found 203 unique rm_id's to focus on.


In [7]:
print("--- Processing kernel_receivals ---")

# Start with a copy to avoid modifying the original dataframe
df_kr_processed = df_kr.copy()

# Filter for relevant rm_id's
initial_rows = len(df_kr_processed)
df_kr_processed = df_kr_processed[df_kr_processed["rm_id"].isin(relevant_rm_ids)]
print(f"Filtered rm_ids: Kept {len(df_kr_processed)} rows out of {initial_rows}.")

# Remove specified columns
df_kr_processed = df_kr_processed.drop(
    columns=["product_id", "receival_status", "batch_id"]
)
print("Removed columns: product_id, receival_status, batch_id.")

# Ensure date_arrival is a datetime object for future joins and feature engineering
df_kr_processed["date_arrival"] = pd.to_datetime(df_kr_processed["date_arrival"])

print("kernel_receivals processing complete.")
df_kr_processed.info()

--- Processing kernel_receivals ---
Filtered rm_ids: Kept 122533 rows out of 122590.
Removed columns: product_id, receival_status, batch_id.
kernel_receivals processing complete.
<class 'pandas.core.frame.DataFrame'>
Index: 122533 entries, 0 to 122589
Data columns (total 7 columns):
 #   Column                  Non-Null Count   Dtype  
---  ------                  --------------   -----  
 0   rm_id                   122533 non-null  float64
 1   purchase_order_id       122533 non-null  float64
 2   purchase_order_item_no  122533 non-null  float64
 3   receival_item_no        122533 non-null  int64  
 4   date_arrival            122533 non-null  object 
 5   net_weight              122520 non-null  float64
 6   supplier_id             122533 non-null  int64  
dtypes: float64(4), int64(2), object(1)
memory usage: 7.5+ MB


/tmp/ipykernel_4245/2481872611.py:16: FutureWarning: In a future version of pandas, parsing datetimes with mixed time zones will raise an error unless `utc=True`. Please specify `utc=True` to opt in to the new behaviour and silence this warning. To create a `Series` with mixed offsets and `object` dtype, please use `apply` and `datetime.datetime.strptime`
  df_kr_processed['date_arrival'] = pd.to_datetime(df_kr_processed['date_arrival'])


In [8]:
print("\n--- Processing kernel_purchase_orders ---")

df_kpo_processed = df_kpo.copy()

# Remove orders with quantity <= 0
initial_rows = len(df_kpo_processed)
df_kpo_processed = df_kpo_processed[df_kpo_processed["quantity"] > 0]
print(
    f"Removed non-positive quantity orders: Kept {len(df_kpo_processed)} rows out of {initial_rows}."
)

# Remove specified columns
df_kpo_processed = df_kpo_processed.drop(
    columns=["product_id", "unit", "unit_id", "status_id", "product_version"]
)
print("Removed columns: product_id, unit, unit_id, status_id, product_version.")

print("kernel_purchase_orders processing complete.")
df_kpo_processed.info()


--- Processing kernel_purchase_orders ---
Removed non-positive quantity orders: Kept 33161 rows out of 33171.
Removed columns: product_id, unit, unit_id, status_id, product_version.
kernel_purchase_orders processing complete.
<class 'pandas.core.frame.DataFrame'>
Index: 33161 entries, 1 to 33170
Data columns (total 7 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   purchase_order_id       33161 non-null  int64  
 1   purchase_order_item_no  33161 non-null  int64  
 2   quantity                33161 non-null  float64
 3   delivery_date           33161 non-null  object 
 4   created_date_time       33161 non-null  object 
 5   modified_date_time      32669 non-null  object 
 6   status                  33161 non-null  object 
dtypes: float64(1), int64(2), object(4)
memory usage: 2.0+ MB


In [9]:
print("\n--- Processing extended_materials ---")

df_em_processed = df_em.copy()

# Filter for relevant rm_id's
df_em_processed = df_em_processed[df_em_processed["rm_id"].isin(relevant_rm_ids)]

# --- Feature Engineering for DELETED stock locations ---

# Identify rows where stock_location indicates deletion
deleted_mask = df_em_processed["stock_location"].str.startswith("DELETED", na=False)
deleted_entries = df_em_processed[deleted_mask]

# Find rm_ids that ONLY have DELETED stock locations
# First, get all rm_ids that have at least one DELETED entry
rm_ids_with_deleted = deleted_entries["rm_id"].unique()

# Now, find which of those rm_ids ALSO have non-DELETED entries
active_locations_for_same_rm_id = df_em_processed[
    (df_em_processed["rm_id"].isin(rm_ids_with_deleted)) & (~deleted_mask)
]
rm_ids_with_mixed_status = active_locations_for_same_rm_id["rm_id"].unique()

# The truly deleted rm_ids are those that are in the deleted list but NOT in the mixed list
truly_deleted_rm_ids = set(rm_ids_with_deleted) - set(rm_ids_with_mixed_status)
print(f"Identified {len(truly_deleted_rm_ids)} rm_ids that are truly deleted.")

# Create the new boolean and date features
df_em_processed["is_deleted"] = df_em_processed["rm_id"].isin(truly_deleted_rm_ids)

# Extract the deleted date (this is a bit tricky, we'll use regex)
# This extracts the YYYY-MM-DD date from strings like "DELETED 2022-01-15 ..."
df_em_processed["deleted_date"] = df_em_processed["stock_location"].str.extract(
    r"(\d{4}-\d{2}-\d{2})"
)
df_em_processed["deleted_date"] = pd.to_datetime(df_em_processed["deleted_date"])

# Only keep the deleted_date for truly deleted materials
df_em_processed.loc[~df_em_processed["is_deleted"], "deleted_date"] = pd.NaT


# Remove specified columns
df_em_processed = df_em_processed.drop(
    columns=["raw_material_alloy", "product_id", "stock_location"]
)
print("Removed columns: raw_material_alloy, product_id, stock_location.")

# We might have multiple rows per rm_id now, so let's keep the most relevant one.
# We can group by rm_id and take the first entry, as the deleted flags are now consistent per rm_id.
df_em_processed = df_em_processed.groupby("rm_id").first().reset_index()


print("extended_materials processing complete.")
df_em_processed.info()


--- Processing extended_materials ---
Identified 29 rm_ids that are truly deleted.
Removed columns: raw_material_alloy, product_id, stock_location.
extended_materials processing complete.
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 203 entries, 0 to 202
Data columns (total 5 columns):
 #   Column                    Non-Null Count  Dtype         
---  ------                    --------------  -----         
 0   rm_id                     203 non-null    float64       
 1   product_version           203 non-null    float64       
 2   raw_material_format_type  203 non-null    float64       
 3   is_deleted                203 non-null    bool          
 4   deleted_date              0 non-null      datetime64[ns]
dtypes: bool(1), datetime64[ns](1), float64(3)
memory usage: 6.7 KB


In [10]:
print("\n--- Processing extended_transportation ---")

df_et_processed = df_et.copy()

# Filter for relevant rm_id's
df_et_processed = df_et_processed[df_et_processed["rm_id"].isin(relevant_rm_ids)]

# Remove specified columns
columns_to_drop = [
    "product_id",
    "unit_status",
    "wood",
    "ironbands",
    "plastic",
    "water",
    "ice",
    "other",
    "chips",
    "packaging",
    "cardboard",
]
df_et_processed = df_et_processed.drop(columns=columns_to_drop)
print(f"Removed columns: {', '.join(columns_to_drop)}.")

print("extended_transportation processing complete.")
df_et_processed.info()


--- Processing extended_transportation ---
Removed columns: product_id, unit_status, wood, ironbands, plastic, water, ice, other, chips, packaging, cardboard.
extended_transportation processing complete.
<class 'pandas.core.frame.DataFrame'>
Index: 122533 entries, 0 to 122589
Data columns (total 12 columns):
 #   Column                  Non-Null Count   Dtype  
---  ------                  --------------   -----  
 0   rm_id                   122533 non-null  float64
 1   purchase_order_id       122533 non-null  float64
 2   purchase_order_item_no  122533 non-null  float64
 3   receival_item_no        122533 non-null  int64  
 4   batch_id                64716 non-null   float64
 5   transporter_name        122533 non-null  object 
 6   vehicle_no              122533 non-null  object 
 7   vehicle_start_weight    122518 non-null  float64
 8   vehicle_end_weight      122496 non-null  float64
 9   gross_weight            122520 non-null  float64
 10  tare_weight             108049 non-n

In [11]:
print("\n--- Joining into master_df ---")

# Use a composite key for joining purchase orders and transportation data
# to handle cases where a single order has multiple items or receivals.
key_cols = ["purchase_order_id", "purchase_order_item_no"]

# Start with receivals as the base (the "fact" table)
master_df = pd.merge(df_kr_processed, df_kpo_processed, on=key_cols, how="left")
print(f"Joined receivals and purchase orders. Shape: {master_df.shape}")

# Merge materials data (on rm_id)
master_df = pd.merge(master_df, df_em_processed, on="rm_id", how="left")
print(f"Joined materials data. Shape: {master_df.shape}")

# Merge transportation data
# Note: Transportation data might have more keys, e.g., receival_item_no.
# For simplicity, we'll join on the primary keys. You may need to refine this join
# if you find it creates duplicates.
transport_key_cols = key_cols + ["rm_id"]
master_df = pd.merge(
    master_df,
    df_et_processed,
    on=transport_key_cols,
    how="left",
    suffixes=("", "_transport"),
)
print(f"Joined transportation data. Shape: {master_df.shape}")


# --- Final Sanity Checks ---
master_df.info()

# --- Save the Processed Data ---
# Saving as parquet is faster and preserves data types better than CSV
# save_data(master_df, "master_data.parquet", folder="2_interim")
# Or use the provided CSV saver
save_data(master_df, "master_data.csv", folder="2_interim")


--- Joining into master_df ---
Joined receivals and purchase orders. Shape: (122533, 12)
Joined materials data. Shape: (122533, 16)
Joined transportation data. Shape: (2782749, 25)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2782749 entries, 0 to 2782748
Data columns (total 25 columns):
 #   Column                      Dtype         
---  ------                      -----         
 0   rm_id                       float64       
 1   purchase_order_id           float64       
 2   purchase_order_item_no      float64       
 3   receival_item_no            int64         
 4   date_arrival                object        
 5   net_weight                  float64       
 6   supplier_id                 int64         
 7   quantity                    float64       
 8   delivery_date               object        
 9   created_date_time           object        
 10  modified_date_time          object        
 11  status                      object        
 12  product_version             

In [13]:
# Order datasets for iteration
df_m = load_data("master_data.csv", "2_interim")

datasets = {
    "master_data": df_m,
}
generate_feature_presence_table(datasets)

Data loaded successfully from /home/kfkh/Code/tdt4173-course-project/data/2_interim/master_data.csv
--- Feature Presence & Fill Grade Across All DataFrames ---
+----------------------------+---------------+
|        Feature Name        |  master_data  |
+============================+===============+
|          batch_id          |   ✅ 31.5%    |
+----------------------------+---------------+
|     created_date_time      |   ✅ 100.0%   |
+----------------------------+---------------+
|        date_arrival        |   ✅ 100.0%   |
+----------------------------+---------------+
|        deleted_date        |    ✅ 0.0%    |
+----------------------------+---------------+
|       delivery_date        |   ✅ 100.0%   |
+----------------------------+---------------+
|        gross_weight        |   ✅ 100.0%   |
+----------------------------+---------------+
|         is_deleted         |   ✅ 100.0%   |
+----------------------------+---------------+
|     modified_date_time     |   ✅ 100.0%   |
+-